In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoSuchWindowException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import random

In [50]:
import pickle

In [29]:
# Example we are trying to scrape text from the blow url. Because Instagram
# requires login we suggest that you auto login using chrome and then use chrome user data
# as an argument to load the chrome browser
# The browser is fully loaded and viewable to the user as this is not a fully automated scraper
# Close any open chrome browser before running or an error is thrown
url = 'https://www.instagram.com/eggsoeufs/'
options = webdriver.ChromeOptions()
options.add_argument(r"--user-data-dir=C:\Users\alexe\AppData\Local\Google\Chrome\User Data")
options.add_argument(r'--profile-directory=Default')
driver = webdriver.Chrome(executable_path=r"chromedriver.exe", options=options)
driver.get(url)

In [14]:
#Using proxy to get to site. Will have to add code to wait until page is loaded to continue
proxy_server_url = "64.226.113.167:8080"
url = 'https://www.instagram.com/eggsoeufs/'
options = webdriver.ChromeOptions()
options.add_argument(r"--user-data-dir=C:\Users\alexe\AppData\Local\Google\Chrome\User Data")
options.add_argument(r'--profile-directory=Default')
options.add_argument(f'--proxy-server={proxy_server_url}')
driver = webdriver.Chrome(executable_path=r"chromedriver.exe", options=options)
driver.get(url)

In [118]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [30]:
# We will auto scroll until either we can't scroll anymore or the number of iterations of the scroll
# reaches 0 based on the count value. Thus if the page has a large number of inputs we will not be able
# to save everything.
# Since we are trying to get the text of the images we are collecting the links and storing them in list_links
last_height = driver.execute_script("return document.body.scrollHeight")
boolScroll = True
count = 200
set_links = set()
while (boolScroll) and (count > 0):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    count = count - 1
    temp = driver.find_elements(By.XPATH,"//div[@class='_aabd _aa8k  _al3l']")
    for item in temp:
        try:
            temp1 = item.find_element(By.TAG_NAME, 'a')
            set_links.add(temp1.get_attribute('href'))
        except StaleElementReferenceException as Exception:
            time.sleep(0.5)
    time.sleep(7)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        boolScroll = False
        print(new_height)
    else:
        last_height = new_height
    print(last_height, end="\r")
driver.quit()
print(count)

148057
858057


In [87]:
print(len(set_links), count)

587 82


In [78]:
df = pd.DataFrame(columns = ['Date', 'Text', 'Poster', 'Likes', 'MultiMedia'])

In [79]:
options = webdriver.ChromeOptions()
options.add_argument(r"--user-data-dir=C:\Users\alexe\AppData\Local\Google\Chrome\User Data")
options.add_argument(r'--profile-directory=Default')
driver = webdriver.Chrome(executable_path=r"chromedriver.exe", options=options)
lst_timeout = np.arange(7,20,1)
lst_failed = []
lst_group_timeout = np.arange(180,240,1)
intTimeout = 0
count = 0
for l in range(len(set_links)):
    count += 1
    print(count, end="\r")
    link = set_links.pop()
    if count%15 == 0:
        intTimeout = lst_group_timeout[random.randint(0,len(lst_group_timeout)-1)]
    else:
        intTimeout = lst_timeout[random.randint(0, len(lst_timeout)-1)]
    try:
        driver.get(link)
        time.sleep(intTimeout)
        date = driver.find_element(By.XPATH,"//time[@class='_aaqe']").get_attribute("title")
        text = driver.find_element(By.XPATH, "//h1[@class='_aacl _aaco _aacu _aacx _aad7 _aade']").text
        by = driver.find_element(By.XPATH, "//div[@class='xt0psk2']").text
        mm_link = driver.find_element(By.XPATH, "//div[@class='_aagv']/img").get_attribute('src')
        try:
            likes = driver.find_element(
                By.XPATH,"//span[@class='x193iq5w xeuugli x1fj9vlw x13faqbe x1vvkbs xt0psk2 x1i0vuye xvs91rp x1s688f x5n08af x10wh9bi x1wdrske x8viiok x18hxmgj']").text
            likes = likes.replace(" likes", "")
        except NoSuchElementException as nse:
            likes = 0
            
        df_temp = pd.DataFrame({'Date' : [date], 'Text' : [text], 'Poster' : [by], 'Likes': [likes],
                                'MultiMedia' : [mm_link]})
        df = pd.concat([df,df_temp])
    except Exception as e:
        print(e)
        print("Page {} failed for some reason. Exiting".format(link))
        lst_failed.append(link)
        continue
driver.quit()
df.shape

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='_aagv']/img"}
  (Session info: chrome=113.0.5672.127)

Page https://www.instagram.com/p/Ckld0F2AHLm/ failed for some reason. Exiting


KeyboardInterrupt: 

In [17]:
# Use this code only if proxy is required
proxy_server_url = "64.226.113.167:8080"
options = webdriver.ChromeOptions()
options.add_argument(r"--user-data-dir=C:\Users\alexe\AppData\Local\Google\Chrome\User Data")
options.add_argument(r'--profile-directory=Default')
#options.add_argument(f'--proxy-server={proxy_server_url}')
driver = webdriver.Chrome(executable_path=r"chromedriver.exe", options=options)
df = pd.DataFrame(columns = ['Title', 'Date', 'Text', 'Poster', 'MultiMedia'])
lst_failed = []
intTimeout = 30
intWait = 10
set_links = ("https://www.instagram.com/p/CWtZi5igaY4/", "https://www.instagram.com/p/Ch-GEDvg3TK/",
            "https://www.instagram.com/p/B87JuPnFxiT/")
count = 0
for link in set_links:
    count += 1
    
    try:
        driver.get(link)
        time.sleep(intWait)
        elem = WebDriverWait(driver, intTimeout).until(EC.presence_of_element_located((By.XPATH, 
                                                                               "//time[@class='_aaqe']")))
        title = elem.get_attribute("title")
        #title = driver.find_element(By.XPATH, "//time[@class='_aaqe']").get_attribute("title")
        elem = WebDriverWait(driver, intTimeout).until(EC.presence_of_element_located((By.XPATH, 
                                                                               "//time[@class='_aaqe']")))
        date = elem.get_attribute("title")
        #date = driver.find_element(By.XPATH,"//time[@class='_aaqe']").get_attribute("title")
        elem = WebDriverWait(driver, intTimeout).until(
            EC.presence_of_element_located((By.XPATH, 
                                            "//h1[@class='_aacl _aaco _aacu _aacx _aad7 _aade']")))
        text = elem.text
        #text = driver.find_element(By.XPATH, "//h1[@class='_aacl _aaco _aacu _aacx _aad7 _aade']").text
        elem = WebDriverWait(driver, intTimeout).until(
            EC.presence_of_element_located((By.XPATH, 
                                            "//div[@class='xt0psk2']")))
        by = elem.text
        #by = driver.find_element(By.XPATH, "//div[@class='xt0psk2']").text
        elem = WebDriverWait(driver, intTimeout).until(
            EC.presence_of_element_located((By.XPATH, 
                                            "//div[@class='_aagv']/img")))
        mm_link = elem.get_attribute('src')
        #mm_link = driver.find_element(By.XPATH, "//div[@class='_aagv']/img").get_attribute('src')
        df_temp = pd.DataFrame({'Title' : [title], 'Date' : [date], 'Text' : [text], 'Poster' : [by], 
                                'MultiMedia' : [mm_link]})
        df = pd.concat([df,df_temp])
    except Exception as e:
        print("Page {} failed because {}".format(link, e))
        lst_failed.append(link)
        continue
    print(count, end="\r")
df.shape

Page https://www.instagram.com/p/Ch-GEDvg3TK/ failed because Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=113.0.5672.94)

Page https://www.instagram.com/p/B87JuPnFxiT/ failed because Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=113.0.5672.94)



(1, 5)

In [80]:
len(set_links), len(lst_failed)

(586, 1)

In [81]:
set_links.update(lst_failed)
len(set_links)

587

In [82]:
with open('set_links.txt','wb') as f:
    pickle.dump(set_links, f)

In [83]:
df.to_csv("Partial.csv",mode='a', header=False, index=False)

In [86]:
df.head()

,Date,Text,Poster,Likes,MultiMedia
0,"Sep 8, 2016",Sweet or savoury? How about both! Get this rec...,eggsoeufs,44,https://scontent.cdninstagram.com/v/t51.2885-1...
0,"Oct 22, 2018",Who said meal prep was just for lunch? Soft Bo...,eggsoeufs,"1,665",https://scontent.cdninstagram.com/v/t51.2885-1...
0,"Feb 2, 2017","This one’s for you, #WiartonWillie! Happy Grou...",eggsoeufs,95,https://scontent.cdninstagram.com/v/t51.2885-1...
0,"Feb 7, 2022",Turn your poached eggs into a success with the...,eggsoeufs,151,https://instagram.fymq2-1.fna.fbcdn.net/v/t51....
0,"Apr 27, 2016",@ShawnMendes has been cracking out one hit alb...,eggsoeufs,47,https://scontent.cdninstagram.com/v/t51.2885-1...


In [85]:
df.shape

(80, 5)

In [23]:
driver.get(list(set_links)[-1])

In [24]:
driver.find_element(By.XPATH,"//time[@class='_aaqe']").get_attribute("title")

'Aug 12, 2022'

In [25]:
driver.find_element(By.XPATH, "//h1[@class='_aacl _aaco _aacu _aacx _aad7 _aade']").text

'My weekends are truly not complete without #BrunchTime🍳 and my brunch meal staple is ALWAY EGGS 🍳. #ad I love all the creative ways our local restaurants use eggs in their menus, from a good old-fashioned classic breakfast to an unforgettable brunch bowl egg-sperience. Here are some of my favourite local spots featuring egg-ceptional egg dishes that you need to check out:\n\n🥚 Classic Breakfast from @bluestar_yyc\n\n🥚Bagel Forestiere from @oeb_breakfast\n\n🥚Truffle bacon mushroom bowl and Duck Confit Monte Cristo from @namoyyc\n\n🥚Bird nest from @zcrewcafe\n\nLet me know your favourite brunch spot in Calgary! @eggsoeufs #EggsAnytime #LocalLove'

In [26]:
driver.find_element(By.XPATH, "//div[@class='xt0psk2']").text

'rachelhaneats'

In [27]:
driver.find_element(By.XPATH, "//div[@class='_aagv']/img").get_attribute('src')

'https://scontent.cdninstagram.com/v/t51.2885-15/298670680_610408290444642_6670465811055762567_n.jpg?stp=dst-jpg_e35&_nc_ht=scontent.cdninstagram.com&_nc_cat=106&_nc_ohc=GG5oz4w6OhsAX_yZklV&edm=APs17CUBAAAA&ccb=7-5&ig_cache_key=MjkwMzQwNjMzMDYyODcyMDM4Mg%3D%3D.2-ccb7-5&oh=00_AfDhg1fDDiA1tizvKV_NNGp_QkvIG5X71Hawd2FG_2k21w&oe=6466866E&_nc_sid=978cb9'